In [2]:
year = 2023
day = 20

In [35]:
from aocd import submit
from aocd.models import Puzzle
from functools import reduce
import numpy as np

puzzle = Puzzle(year=year, day=day)
data = puzzle.input_data

data = data.strip()
data = data.split("\n")
data = {k[1:].strip(): (k[0], [i.strip() for i in v.split(",")]) for k, v in (d.split("->") for d in data)}

In [36]:
from collections import defaultdict
conjunction_mem = defaultdict(lambda: defaultdict(int))
for k, v in data.items():
    for n in v[1]:
        if n == "output" or n == "rx":
            continue
        if data[n][0] == "&":
            conjunction_mem[n][k] = 0

In [37]:
inv_mem = {k: 0 for k, v in data.items() if v[0] == "%"}

In [38]:
def reset_inv_mem():
    for k in inv_mem:
        inv_mem[k] = 0
    return inv_mem

def reset_conjunction_mem():
    for k in conjunction_mem:
        for k2 in conjunction_mem[k]:
            conjunction_mem[k][k2] = 0
    return conjunction_mem

In [48]:
final_node = [k for k, v in data.items() if "rx" in v[1]][0]
final_node

'tj'

In [50]:
# %: flip flop
# b: broadcast
# &: 'and' of all inputs

pulse_list = [("button", "roadcaster", 0)]
MAX_ITER = 10_000

# def simulate(pulse_list, inv_mem, conjunction_mem):

inv_mem = reset_inv_mem()
conjunction_mem = reset_conjunction_mem()
total_pulses_high = 0
total_pulses_low = 0

pulses_per_round = []
cycle_values = {}

# 5000 was empirically determined to be enough to see all cycles
for round in range(1, 5_000):

    pulse_list = [("button", "roadcaster", 0)]

    pulses_high = 0
    pulses_low = 1 # initial pulse from button
    print_round = False
    for i in range(MAX_ITER):
        new_pulse_list = []
        for sender, node, val in pulse_list:
            if node not in data:
                continue
            if node == "rx":
                print("rx", val)
            node_type, connections = data[node]
            if node_type == "b":
                new_pulses = [(node, n, 0) for n in connections]
            elif node_type == "%":
                if not val:
                    inv_mem[node] = 1 - inv_mem[node]
                    new_pulses = [(node, n, inv_mem[node]) for n in connections]
                else:
                    new_pulses = []
            elif node_type == "&":
                conjunction_mem[node][sender] = val
                # print(node, conjunction_mem[node], all(conjunction_mem[node].values()))
                output = 0 if all(conjunction_mem[node].values()) else 1
                new_pulses = [(node, n, output) for n in connections]
            else:
                raise ValueError
            new_pulse_list.extend(new_pulses)
        # for p in new_pulse_list:
        #     print(f"{p[0]} -{'high' if p[2] else 'low'}-> {p[1]}")
        pulse_list = new_pulse_list
        pulses_high += sum(p[2] for p in pulse_list)
        pulses_low += sum(1 - p[2] for p in pulse_list)
        if any(conjunction_mem[final_node].values()) and not print_round:
            print(round, i, conjunction_mem[final_node].values())
            if conjunction_mem[final_node].values() not in cycle_values:
                cycle_values[conjunction_mem[final_node].values()] = round
            print_round = True
        if not pulse_list:
            break
    else:
        raise ValueError("Too many iterations")
    pulses_per_round.append((pulses_high, pulses_low))

    total_pulses_high += pulses_high
    total_pulses_low += pulses_low

    if round == 1000:
        answer = total_pulses_high*total_pulses_low

3917 4 dict_values([0, 1, 0, 0])
3931 4 dict_values([0, 0, 0, 1])
3943 4 dict_values([1, 0, 0, 0])
4057 4 dict_values([0, 0, 1, 0])


In [51]:
submit(answer, part="a", day=day, year=year)

Part a already solved with same answer: 818649769


In [57]:
import math
# empirically determined that all inputs of final_node cycle and are prime
# so the answer is the product of the cycle values
answer = math.prod(cycle_values.values())
submit(answer, part="b", day=day, year=year)

Part b already solved with same answer: 246313604784977
